## Servidor FastAPI para el modelo de Supervivencia del Titanic

En este notebook levantamos un **servidor FastAPI** que carga el artefacto entrenado y expone un endpoint de predicción.

**Qué hace:**
- Carga `model/logistic_titanic_pipeline.pkl` (pipeline + threshold + features).
- Define un esquema `Passenger` (Pydantic) para validar entradas.
- Expone:
  - `GET /` → ping básico.
  - `GET /healthz` → health check (lo usa Render).
  - `POST /predict` → recibe un JSON con los campos del pasajero y devuelve:
    - `prob_survive` (probabilidad),
    - `survived` (True/False según umbral),
    - `threshold_used`.

**Cómo probar local:**
1. Ejecuta las celdas para definir la app.
2. (En terminal, recomendado para desarrollo) desde la raíz del repo:
   ```bash
   uvicorn app.main:app --reload


Luego visita http://127.0.0.1:8000/docs y prueba POST /predict.

### Cargar dependencias y artefacto del modelo

En esta sección:

- **Imports**: librerías de FastAPI, Pydantic, `joblib`, `pandas`, etc.  
  - `nest_asyncio` + `uvicorn` permiten **levantar el servidor dentro del notebook** (útil para prues de columnas.


In [12]:
# === 01_server_titanic.ipynb ===
# Servidor FastAPI para el modelo de supervivencia del Titanic

import os
import joblib
import pandas as pd
import numpy as np

from typing import Optional, Literal

import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field

- **Ruta del artefacto**: `ARTIFACT_PATH = "model/logistic_titanic_pipeline.pkl"`.
- **Chequeo de existencia**: si el archivo no está, se lanza un `FileNotFoundError` indicando que primero hay que entrenar/guardar en el notebook `00`.
- **Carga con `joblib`**: el artefacto es un **diccionario** con:
  - `model`: Pipeline de scikit-learn ya entrenado (posee `.predict_proba`).
  - `threshold`: umbral recomendado para decidir la clase.
  - `features`: columnas crudas esperadas por el pipeline (y su **orden**).
- **Valores por defecto** (fallback): si no vinieran los metadatos, se usa `threshold=0.5` y  
  `features = ['pclass','sex','age','sibsp','parch','fare','embarked']`.

In [13]:
# -----------------------------
# 1) Cargar artefacto del modelo
# -----------------------------
ARTIFACT_PATH = "model/logistic_titanic_pipeline.pkl"
if not os.path.exists(ARTIFACT_PATH):
    raise FileNotFoundError(
        f"No se encontró {ARTIFACT_PATH}. Entrena y guarda el modelo en el notebook 00 primero."
    )

artifact = joblib.load(ARTIFACT_PATH)
pipe = artifact["model"]
DEFAULT_THRESHOLD = float(artifact.get("threshold", 0.5))
EXPECTED_FEATURES = artifact.get(
    "features",
    ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']  # fallback
)

### Definición de la API (FastAPI)

En esta sección se crea la **aplicación FastAPI** y se definen los **endpoints** y el
**esquema de entrada**:

- `app = FastAPI(...)`: metadatos básicos (título, versión, descripción).
- **Pydantic model `Passenger`**  
  Valida y documenta el JSON de entrada:
  - `pclass ∈ {1,2,3}`, `sex ∈ {"male","female"}`, `age ∈ [0,100]` (opcional),  
    `sibsp ≥ 0`, `parch ≥ 0`, `fare ≥ 0` (opcional), `embarked ∈ {"C","Q","S"}` (opcional).
  - `to_dataframe()` normaliza (`sex` a minúsculas, `embarked` a mayúsculas) y
    **garantiza el orden de columnas** esperado por el pipeline (`EXPECTED_FEATURES`).
- **Endpoints**
  - `GET /` → mensaje de bienvenida + `default_threshold` + `expected_features`.
  - `GET /healthz` → *health check* simple (`{"status": "healthy"}`).
  - `POST /predict` → recibe un `Passenger` y devuelve:
    - `prob_survive`: probabilidad de sobrevivir (clase positiva).
    - `threshold_used`: umbral aplicado (por defecto `DEFAULT_THRESHOLD`).
    - `pred_class` (`0|1`) y `pred_label` (“Sobrevive” / “No sobrevive”).
    - `features_order`: columnas usadas por el modelo.
  - **Query param opcional**: `confidence` permite **forzar** el umbral de decisión.
- **Errores**: cualquier inconsistencia levanta `HTTPException(400)` con el detalle.

**Ejemplo de JSON válido**
```json
{
  "pclass": 1,
  "sex": "female",
  "age": 22,
  "sibsp": 0,
  "parch": 1,
  "fare": 80.0,
  "embarked": "C"
}


In [14]:
# -----------------------------
# 2) Definición de la API
# -----------------------------
app = FastAPI(
    title="API - Supervivencia Titanic",
    description="Clasificador binario de supervivencia usando Pipeline de scikit-learn.",
    version="1.0.0"
)

# Schema de entrada (campos crudos; el Pipeline se encarga del preprocesamiento)
class Passenger(BaseModel):
    pclass: Literal[1, 2, 3] = Field(..., description="Clase del boleto (1, 2, 3)")
    sex: Literal["male", "female"] = Field(..., description="Sexo del pasajero")
    age: Optional[float] = Field(None, ge=0, le=100, description="Edad en años (puede ser nula)")
    sibsp: int = Field(..., ge=0, description="Hnos/cónyuges a bordo")
    parch: int = Field(..., ge=0, description="Padres/hijos a bordo")
    fare: Optional[float] = Field(None, ge=0, description="Tarifa pagada (puede ser nula)")
    embarked: Optional[Literal["C", "Q", "S"]] = Field(None, description="Puerto de embarque")

    # normalizamos por si vienen en minúsculas / mayúsculas mezcladas
    def to_dataframe(self) -> pd.DataFrame:
        row = {
            "pclass": int(self.pclass),
            "sex": str(self.sex).lower(),
            "age": self.age,
            "sibsp": int(self.sibsp),
            "parch": int(self.parch),
            "fare": self.fare,
            "embarked": None if self.embarked is None else str(self.embarked).upper(),
        }
        # asegurar orden de columnas esperado por el pipeline
        return pd.DataFrame([row], columns=EXPECTED_FEATURES)


@app.get("/")
def home():
    return {
        "message": "¡API Titanic OK! Visita /docs para probar.",
        "default_threshold": DEFAULT_THRESHOLD,
        "expected_features": EXPECTED_FEATURES,
    }


@app.get("/healthz")
def healthz():
    return {"status": "healthy"}


@app.post("/predict")
def predict(passenger: Passenger, confidence: Optional[float] = None):
    """
    Predice supervivencia para un pasajero.
    - Usa 'confidence' como umbral si se especifica; de lo contrario usa el umbral guardado.
    """
    try:
        X = passenger.to_dataframe()
        # Probabilidad de clase positiva (sobrevive = 1)
        proba = float(pipe.predict_proba(X)[0, 1])
        thr = float(confidence) if confidence is not None else DEFAULT_THRESHOLD
        pred = int(proba >= thr)
        label = "Sobrevive" if pred == 1 else "No sobrevive"
        return {
            "threshold_used": thr,
            "prob_survive": proba,
            "pred_class": int(pred),
            "pred_label": label,
            "features_order": EXPECTED_FEATURES,
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error en predicción: {e}")

### Ejecutar el servidor desde el notebook

Para hacer una **demo rápida** del API dentro de Jupyter:

- `nest_asyncio.apply()` permite que **Uvicorn** corra en el **event loop** de Jupyter (sin esto, suele fallar con “event loop already running”).
- `uvicorn.run(app, host="127.0.0.1", port=8000)` levanta el servidor local en `http://127.0.0.1:8000`.

Luego prueba:
- `http://127.0.0.1:8000/healthz`
- `http://127.0.0.1:8000/docs` (Swagger)

> Nota: esta celda **bloquea** la ejecución mientras el servidor esté arriba. Para detenerlo, interrumpe el kernel.

**Producción / terminal**
- Local (fuera del notebook):  
  `uvicorn app.main:app --reload`
- Render (o contenedor):  
  `uvicorn app.main:app --host 0.0.0.0 --port $PORT`


In [ ]:
# -----------------------------
# 3) Ejecutar servidor (en notebook)
# -----------------------------

nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [27308]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:55017 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55018 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55019 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55020 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55237 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55240 - "GET /predict?confidence=0.55 HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:55245 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:55246 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:55246 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:55249 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:60698 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60698 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60701 - "GET /healthz HTTP/1.1" 200 OK
INFO:     127.0.0.1:61405 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61405 - "GET /openapi.json HTTP/1.1" 200 OK